# Step 3 - Second Part
Preprocess the dataset done in previous steps. Adjust for any modifications that came out of
optimizing the model. Design a neural network model and adjust for modifications that came 
out of optimizing the model.

Run this in Google Colab

## Preprocessing

In [1]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

In [2]:
# Import and read in the csv file to a Pandas DataFrame
application_df = pd.read_csv("Resources/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [3]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
new_application_df = application_df.drop(columns=["EIN", "NAME"])
new_application_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [4]:
# Determine the number of unique values in each column.
unique_values = new_application_df.nunique(axis=0)
unique_values.head()

APPLICATION_TYPE    17
AFFILIATION          6
CLASSIFICATION      71
USE_CASE             5
ORGANIZATION         4
dtype: int64

In [5]:
# Look at APPLICATION_TYPE value counts for binning - getting the columns you need and grouping the rest to like other. 
app_values = new_application_df.value_counts(["APPLICATION_TYPE"])
app_values

APPLICATION_TYPE
T3                  27037
T4                   1542
T6                   1216
T5                   1173
T19                  1065
T8                    737
T7                    725
T10                   528
T9                    156
T13                    66
T12                    27
T2                     16
T14                     3
T25                     3
T29                     2
T15                     2
T17                     1
dtype: int64

In [6]:
# Choose a cutoff value and create a list of application types to be replaced
# Use the variable name `application_types_to_replace`
application_types_to_replace = list(app_values[app_values < 300].index)

# Replace in dataframe
for app in application_types_to_replace:
    application_df["APPLICATION_TYPE"] = application_df["APPLICATION_TYPE"].replace(app,"Other")

# Check to make sure binning was successful
application_df["APPLICATION_TYPE"].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [7]:
# Look at CLASSIFICATION value counts for binning
class_value = new_application_df.value_counts(["CLASSIFICATION"])
class_value.head(20)

CLASSIFICATION
C1000             17326
C2000              6074
C1200              4837
C3000              1918
C2100              1883
C7000               777
C1700               287
C4000               194
C5000               116
C1270               114
C2700               104
C2800                95
C7100                75
C1300                58
C1280                50
C1230                36
C1400                34
C2300                32
C7200                32
C1240                30
dtype: int64

In [8]:
# Look at CLASSIFICATION value counts >1
class_value[class_value>1]

CLASSIFICATION
C1000             17326
C2000              6074
C1200              4837
C3000              1918
C2100              1883
C7000               777
C1700               287
C4000               194
C5000               116
C1270               114
C2700               104
C2800                95
C7100                75
C1300                58
C1280                50
C1230                36
C1400                34
C2300                32
C7200                32
C1240                30
C8000                20
C7120                18
C1500                16
C1800                15
C6000                15
C1250                14
C8200                11
C1278                10
C1238                10
C1235                 9
C1237                 9
C7210                 7
C1720                 6
C4100                 6
C2400                 6
C1600                 5
C1257                 5
C2710                 3
C1260                 3
C0                    3
C1267                 2
C

In [9]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace = list(class_value[class_value < 300].index)

# Replace in dataframe
for cls in classifications_to_replace:
    application_df["CLASSIFICATION"] = application_df["CLASSIFICATION"].replace(cls,"Other")
    
# Check to make sure binning was successful
application_df["CLASSIFICATION"].value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
Other     1484
C7000      777
Name: CLASSIFICATION, dtype: int64

In [10]:
# Convert categorical data to numeric with `pd.get_dummies`
x_dummies = pd.get_dummies(new_application_df)
print(x_dummies.columns)

Index(['STATUS', 'ASK_AMT', 'IS_SUCCESSFUL', 'APPLICATION_TYPE_T10',
       'APPLICATION_TYPE_T12', 'APPLICATION_TYPE_T13', 'APPLICATION_TYPE_T14',
       'APPLICATION_TYPE_T15', 'APPLICATION_TYPE_T17', 'APPLICATION_TYPE_T19',
       ...
       'INCOME_AMT_1-9999', 'INCOME_AMT_10000-24999',
       'INCOME_AMT_100000-499999', 'INCOME_AMT_10M-50M', 'INCOME_AMT_1M-5M',
       'INCOME_AMT_25000-99999', 'INCOME_AMT_50M+', 'INCOME_AMT_5M-10M',
       'SPECIAL_CONSIDERATIONS_N', 'SPECIAL_CONSIDERATIONS_Y'],
      dtype='object', length=117)


In [11]:
# Split our preprocessed data into our features and target arrays 
y = x_dummies["IS_SUCCESSFUL"].values
X = x_dummies.drop(["IS_SUCCESSFUL"], axis=1).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [12]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [14]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_features = len(X_train[0])
hidden_nodes_layer1 =  7
hidden_nodes_layer2 = 5

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, activation="relu", input_dim=number_features))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 7)                 819       
                                                                 
 dense_4 (Dense)             (None, 5)                 40        
                                                                 
 dense_5 (Dense)             (None, 1)                 6         
                                                                 
Total params: 865
Trainable params: 865
Non-trainable params: 0
_________________________________________________________________


In [15]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [16]:
# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
804/804 [==============================] - 3s 3ms/step - loss: 0.6131 - accuracy: 0.6996
Epoch 2/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5609 - accuracy: 0.7278
Epoch 3/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5549 - accuracy: 0.7305
Epoch 4/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5524 - accuracy: 0.7314
Epoch 5/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5511 - accuracy: 0.7316
Epoch 6/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5502 - accuracy: 0.7327
Epoch 7/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5492 - accuracy: 0.7318
Epoch 8/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5487 - accuracy: 0.7326
Epoch 9/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5484 - accuracy: 0.7321
Epoch 10/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5474 - accura

In [17]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5594 - accuracy: 0.7237 - 445ms/epoch - 2ms/step
Loss: 0.5593581199645996, Accuracy: 0.7237317562103271


In [18]:
# Create a DataFrame containing training history
history_df = pd.DataFrame(fit_model.history)

In [19]:
# Export our model to HDF5 file
nn.save('HDF5/AlphabetSoupCharity_Optimization.h5')